In [ ]:
!gdown 1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
!gdown 1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1

Downloading...
From: https://drive.google.com/uc?id=1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
To: /content/test_data.csv
100% 565k/565k [00:00<00:00, 87.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1
To: /content/train_data.csv
100% 2.24M/2.24M [00:00<00:00, 141MB/s]


In [ ]:
import numpy as np
import pandas as pd
import gensim
import gensim.downloader as api
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [ ]:
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")

In [ ]:
train_df = train_df[train_df['confidence'] >= 0.8].reset_index(drop=True)

In [ ]:
train_df.shape

(1832, 111)

In [ ]:
test_df = test_df[test_df['confidence'] >= 0.8].reset_index(drop=True)
test_df.shape

(465, 111)

In [ ]:
X_train = train_df['clean_text'].to_numpy()
y_train = train_df['target'].to_numpy()

In [ ]:
X_test = test_df['clean_text'].to_numpy()
y_test = test_df['target'].to_numpy()

# Pretrained Word2Vec

In [ ]:
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

[==================================================] 100.0% 1662.8/1662.8MB downloaded
/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [ ]:
model_w2v = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
voc = set(model_w2v.index_to_key)
def document_vector(doc):
    doc = doc.split()
    doc = [word for word in doc if word in voc]
    return np.mean(model_w2v[doc], axis=0)

In [ ]:
X_train_w2v = []
for review in X_train:
    X_train_w2v.append(document_vector(review))

In [ ]:
X_test_w2v = []
for review in X_test:
    X_test_w2v.append(document_vector(review))

In [ ]:
np.array(X_train_w2v).shape

(1832, 300)

# Training and Testing

In [ ]:
def training_and_prediction(training_model, X_train, X_test, y_train, y_test):
  model = training_model
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  # print("Accuracy =", accuracy)
  # print("Confusion Matrix =\n", confusion_matrix(y_test, y_pred))
  # print(classification_report(y_test, y_pred))
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

In [ ]:
def get_report(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

## Naive Bayes

In [ ]:
scaler = MinMaxScaler()
training_and_prediction(MultinomialNB(), scaler.fit_transform(X_train_w2v), scaler.fit_transform(X_test_w2v), y_train, y_test)

{'Accuracy': 0.7548387096774194,
 'Precision': 0.78125,
 'Recall': 0.7291666666666666,
 'F1-Score': 0.7543103448275862}

In [ ]:
training_and_prediction(GaussianNB(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7182795698924731,
 'Precision': 0.7026022304832714,
 'Recall': 0.7875,
 'F1-Score': 0.7426326129666012}

In [ ]:
training_and_prediction(BernoulliNB(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7290322580645161,
 'Precision': 0.7065217391304348,
 'Recall': 0.8125,
 'F1-Score': 0.7558139534883721}

## SVM

In [ ]:
training_and_prediction(SVC(kernel='poly', C=6), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7978494623655914,
 'Precision': 0.7896825396825397,
 'Recall': 0.8291666666666667,
 'F1-Score': 0.8089430894308943}

## Random Forest

In [ ]:
training_and_prediction(RandomForestClassifier(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7763440860215054,
 'Precision': 0.7635658914728682,
 'Recall': 0.8208333333333333,
 'F1-Score': 0.7911646586345381}

## Logistic Regression

In [ ]:
training_and_prediction(LogisticRegression(solver='saga'), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7870967741935484,
 'Precision': 0.7786561264822134,
 'Recall': 0.8208333333333333,
 'F1-Score': 0.7991886409736308}

## Gradient Boosting

In [ ]:
training_and_prediction(XGBClassifier(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7548387096774194,
 'Precision': 0.7540322580645161,
 'Recall': 0.7791666666666667,
 'F1-Score': 0.7663934426229508}

## Deep Learning Techniques with Word2vec

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Bidirectional, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Tokenize text data
tokenizer = Tokenizer(num_words=7000)
tokenizer.fit_on_texts(np.concatenate((X_train, X_test), axis=0))
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
high = 0
for temp in X_train_sequences:
  high = max(high, len(temp))
high

141

In [ ]:
# Pad sequences to same length
max_len = 150
X_train_padded_sequences = pad_sequences(X_train_sequences, maxlen=max_len)
X_test_padded_sequences = pad_sequences(X_test_sequences, maxlen=max_len)

In [ ]:
# Create embedding layer
embedding_dim = 300
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in model_w2v:
        embedding_matrix[i] = model_w2v[word]
embedding_layer = Embedding(len(tokenizer.word_index) + 1, embedding_dim, weights=[embedding_matrix], trainable=False)

### ANN

In [ ]:
def ann_models(num_neurons_layer1, num_neurons_layer2=0, dropout_rate=0):
    model_dense = Sequential()
    model_dense.add(embedding_layer)
    model_dense.add(GlobalAveragePooling1D())
    model_dense.add(Dense(num_neurons_layer1, activation='relu'))
    model_dense.add(Dropout(dropout_rate))
    if num_neurons_layer2 > 0:
      model_dense.add(Dense(num_neurons_layer2, activation='relu'))
    model_dense.add(Dense(1, activation='sigmoid'))
    model_dense.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_dense.fit(X_train_padded_sequences, y_train, epochs=100, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_padded_sequences, y_test))
    y_pred_dense = model_dense.predict(X_test_padded_sequences)
    y_pred_dense = (y_pred_dense > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'num_neurons_layer2': num_neurons_layer2, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_dense)]


In [ ]:
for num_neurons_layer1 in [128, 64]:
  for num_neurons_layer2 in [64, 32, 0]:
    for dropout_rate in [0.0, 0.1, 0.2, 0.3, 0.4]:
      print(ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate))

15/15 [==============================] - 0s 4ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.0}, {'Accuracy': 0.7806451612903226, 'Precision': 0.7674418604651163, 'Recall': 0.825, 'F1-Score': 0.7951807228915662}]
15/15 [==============================] - 0s 3ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.1}, {'Accuracy': 0.7827956989247312, 'Precision': 0.7683397683397684, 'Recall': 0.8291666666666667, 'F1-Score': 0.7975951903807617}]
15/15 [==============================] - 0s 4ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.2}, {'Accuracy': 0.7870967741935484, 'Precision': 0.7680608365019012, 'Recall': 0.8416666666666667, 'F1-Score': 0.8031809145129225}]
15/15 [==============================] - 0s 4ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.3}, {'Accuracy': 0.7763440860215054, 'Precision': 0.765625, 'Recall': 0.8166666666666667, 'F1-Score': 0.790322

### LSTM

In [ ]:
def lstm_models(num_neurons_layer1, dropout_rate=0):
    model_lstm = Sequential()
    model_lstm.add(embedding_layer)
    model_lstm.add(LSTM(128, dropout=dropout_rate))
    model_lstm.add(Dense(1, activation='sigmoid'))
    model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model_lstm.fit(X_train_padded_sequences, y_train, epochs=20, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_padded_sequences, y_test))
    y_pred_lstm = model_lstm.predict(X_test_padded_sequences)
    y_pred_lstm = (y_pred_lstm > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_lstm)]

In [ ]:
for num_neurons_layer1 in [128, 64]:
  for dropout_rate in [0.0, 0.1, 0.2, 0.3, 0.4]:
    print(lstm_models(num_neurons_layer1, dropout_rate))

15/15 [==============================] - 2s 87ms/step
[{'num_neurons_layer1': 128, 'dropout_rate': 0.0}, {'Accuracy': 0.7849462365591398, 'Precision': 0.7941176470588235, 'Recall': 0.7875, 'F1-Score': 0.7907949790794979}]
15/15 [==============================] - 2s 91ms/step
[{'num_neurons_layer1': 128, 'dropout_rate': 0.1}, {'Accuracy': 0.7870967741935484, 'Precision': 0.7949790794979079, 'Recall': 0.7916666666666666, 'F1-Score': 0.7933194154488519}]
15/15 [==============================] - 2s 88ms/step
[{'num_neurons_layer1': 128, 'dropout_rate': 0.2}, {'Accuracy': 0.7655913978494624, 'Precision': 0.7314487632508834, 'Recall': 0.8625, 'F1-Score': 0.7915869980879541}]
15/15 [==============================] - 2s 89ms/step
[{'num_neurons_layer1': 128, 'dropout_rate': 0.3}, {'Accuracy': 0.7741935483870968, 'Precision': 0.7547169811320755, 'Recall': 0.8333333333333334, 'F1-Score': 0.7920792079207922}]
15/15 [==============================] - 2s 86ms/step
[{'num_neurons_layer1': 128, 'drop

### Bidirectional LSTM

In [ ]:
def bilstm_models(num_neurons_layer1, spatial_dropout_rate=0, dropout_rate=0):
    model_bilstm = Sequential()
    model_bilstm.add(embedding_layer)
    model_bilstm.add(SpatialDropout1D(spatial_dropout_rate))
    model_bilstm.add(Bidirectional(LSTM(num_neurons_layer1, return_sequences=False)))
    model_bilstm.add(Dropout(dropout_rate))
    model_bilstm.add(Dense(1, activation='sigmoid'))
    model_bilstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model_bilstm.fit(X_train_padded_sequences, y_train, epochs=10, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_padded_sequences, y_test))
    y_pred_bilstm = model_bilstm.predict(X_test_padded_sequences)
    y_pred_bilstm = (y_pred_bilstm > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'spatial_dropout_rate': spatial_dropout_rate, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_bilstm)]

In [42]:
for num_neurons_layer1 in [128, 64]:
  for spatial_dropout_rate in [0, 0.2]:
    for dropout_rate in [0.1, 0.2, 0.3, 0.4]:
      print(bilstm_models(num_neurons_layer1, spatial_dropout_rate, dropout_rate))

15/15 [==============================] - 6s 211ms/step
[{'num_neurons_layer1': 128, 'spatial_dropout_rate': 0, 'dropout_rate': 0.1}, {'Accuracy': 0.7784946236559139, 'Precision': 0.7665369649805448, 'Recall': 0.8208333333333333, 'F1-Score': 0.7927565392354124}]
15/15 [==============================] - 3s 152ms/step
[{'num_neurons_layer1': 128, 'spatial_dropout_rate': 0, 'dropout_rate': 0.2}, {'Accuracy': 0.7698924731182796, 'Precision': 0.7285223367697594, 'Recall': 0.8833333333333333, 'F1-Score': 0.7984934086629001}]
15/15 [==============================] - 4s 232ms/step
[{'num_neurons_layer1': 128, 'spatial_dropout_rate': 0, 'dropout_rate': 0.3}, {'Accuracy': 0.7677419354838709, 'Precision': 0.7171052631578947, 'Recall': 0.9083333333333333, 'F1-Score': 0.8014705882352942}]
15/15 [==============================] - 4s 211ms/step
[{'num_neurons_layer1': 128, 'spatial_dropout_rate': 0, 'dropout_rate': 0.4}, {'Accuracy': 0.7677419354838709, 'Precision': 0.7682926829268293, 'Recall': 0.787

### GRU

In [43]:
model_gru = Sequential()
model_gru.add(embedding_layer)
# model_gru.add(SpatialDropout1D(0.2))
model_gru.add(GRU(64, return_sequences = False))
model_gru.add(Dropout(0.2))
model_gru.add(Dense(1, activation = 'sigmoid'))
model_gru.summary()

Model: "sequential_56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 300)         2455800   
                                                                 
 gru (GRU)                   (None, 64)                70272     
                                                                 
 dropout_46 (Dropout)        (None, 64)                0         
                                                                 
 dense_106 (Dense)           (None, 1)                 65        
                                                                 
Total params: 2526137 (9.64 MB)
Trainable params: 70337 (274.75 KB)
Non-trainable params: 2455800 (9.37 MB)
_________________________________________________________________


In [44]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_gru.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_gru.fit(X_train_padded_sequences, y_train, epochs=50, batch_size=32, validation_data=(X_test_padded_sequences, y_test), callbacks=[early_stop])

Epoch 1/50
58/58 [==============================] - 12s 162ms/step - loss: 0.6526 - accuracy: 0.5993 - val_loss: 0.5586 - val_accuracy: 0.6989
Epoch 2/50
58/58 [==============================] - 8s 130ms/step - loss: 0.4614 - accuracy: 0.7806 - val_loss: 0.4567 - val_accuracy: 0.7849
Epoch 3/50
58/58 [==============================] - 9s 153ms/step - loss: 0.3940 - accuracy: 0.8286 - val_loss: 0.5120 - val_accuracy: 0.7634
Epoch 4/50
58/58 [==============================] - 9s 154ms/step - loss: 0.3728 - accuracy: 0.8428 - val_loss: 0.4649 - val_accuracy: 0.7806
Epoch 5/50
58/58 [==============================] - 8s 130ms/step - loss: 0.3435 - accuracy: 0.8532 - val_loss: 0.4515 - val_accuracy: 0.7720
Epoch 6/50
58/58 [==============================] - 9s 152ms/step - loss: 0.3321 - accuracy: 0.8570 - val_loss: 0.4659 - val_accuracy: 0.7914
Epoch 7/50
58/58 [==============================] - 9s 155ms/step - loss: 0.2870 - accuracy: 0.8854 - val_loss: 0.4623 - val_accuracy: 0.7720
Epoch

In [45]:
y_pred_gru = model_gru.predict(X_test_padded_sequences)
y_pred_gru = (y_pred_gru > 0.5).astype(int)
get_report(y_test, y_pred_gru)

15/15 [==============================] - 1s 43ms/step


{'Accuracy': 0.7720430107526882,
 'Precision': 0.7887931034482759,
 'Recall': 0.7625,
 'F1-Score': 0.7754237288135593}